In [1]:
"""
approach 01:

dataset X,y 
-> preprocessing.fit(X), preprocessing.transform(X) OR preprocessing.fit_transform(X) 
-> split into training (X_train, y_train) and test (X_test, y_test) sets  
-> model.fit(X_train, y_train) 
-> model.predict(X_test)

"""

'\napproach 01:\n\ndataset X,y \n-> preprocessing.fit(X), preprocessing.transform(X) OR preprocessing.fit_transform(X) \n-> split into training (X_train, y_train) and test (X_test, y_test) sets  \n-> model.fit(X_train, y_train) \n-> model.predict(X_test)\n\n'

## Prerequisites and Loading dataset

In [2]:
# to remove warnings notifications from Scikit-Learn

import warnings
warnings.filterwarnings("ignore")

In [3]:
import numpy as np
import pandas as pd

# to convert ? as NaN
df = pd.read_csv("../input/autism_screening.csv", na_values=['?'])

## Exploratory Data Analysis (EDA)

In [4]:
df.head(3)

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,...,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
0,1,1,1,1,0,0,1,1,0,0,...,f,White-European,no,no,United States,no,6.0,18 and more,Self,NO
1,1,1,0,1,0,0,0,1,0,1,...,m,Latino,no,yes,Brazil,no,5.0,18 and more,Self,NO
2,1,1,0,1,1,0,1,1,1,1,...,m,Latino,yes,yes,Spain,no,8.0,18 and more,Parent,YES


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   A1_Score         704 non-null    int64  
 1   A2_Score         704 non-null    int64  
 2   A3_Score         704 non-null    int64  
 3   A4_Score         704 non-null    int64  
 4   A5_Score         704 non-null    int64  
 5   A6_Score         704 non-null    int64  
 6   A7_Score         704 non-null    int64  
 7   A8_Score         704 non-null    int64  
 8   A9_Score         704 non-null    int64  
 9   A10_Score        704 non-null    int64  
 10  age              702 non-null    float64
 11  gender           704 non-null    object 
 12  ethnicity        609 non-null    object 
 13  jundice          704 non-null    object 
 14  austim           704 non-null    object 
 15  contry_of_res    704 non-null    object 
 16  used_app_before  704 non-null    object 
 17  result          

In [6]:
df.loc[ df.age.isnull()==True ] 

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,...,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
62,0,0,0,0,0,0,0,0,0,0,...,m,NaN,no,no,Saudi Arabia,no,0.0,18 and more,NaN,NO
91,0,1,0,0,1,0,1,0,0,1,...,f,NaN,no,no,Jordan,no,4.0,18 and more,NaN,NO


In [7]:
null_rows_idx = df.isnull().any(axis=1)
df.loc[null_rows_idx]

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,...,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
4,1,0,0,0,0,0,0,1,0,0,...,f,NaN,no,no,Egypt,no,2.0,18 and more,NaN,NO
12,0,1,1,1,1,1,0,0,1,0,...,f,NaN,no,no,Bahamas,no,6.0,18 and more,NaN,NO
13,1,0,0,0,0,0,1,1,0,1,...,m,NaN,no,no,Austria,no,4.0,18 and more,NaN,NO
14,1,0,0,0,0,0,1,1,0,1,...,f,NaN,no,no,Argentina,no,4.0,18 and more,NaN,NO
19,0,0,0,0,0,0,1,1,0,1,...,m,NaN,yes,no,United Arab Emirates,no,3.0,18 and more,NaN,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,0,0,0,0,0,0,0,0,0,1,...,f,NaN,no,no,United States,no,1.0,18 and more,NaN,NO
658,0,0,1,1,0,0,1,0,0,0,...,m,NaN,no,no,Azerbaijan,no,3.0,18 and more,NaN,NO
659,1,1,1,1,1,1,0,0,1,1,...,m,NaN,no,no,Pakistan,no,8.0,18 and more,NaN,YES
666,0,0,0,0,0,0,0,1,0,0,...,m,NaN,no,no,Iraq,no,1.0,18 and more,NaN,NO


In [8]:
df.shape

(704, 21)

In [9]:
target_name = "Class/ASD"
y = df[target_name]

X = df.drop( columns = [target_name, "result"] )

In [10]:
X.shape

(704, 19)

In [11]:
y

0       NO
1       NO
2      YES
3       NO
4       NO
      ... 
699    YES
700     NO
701    YES
702     NO
703    YES
Name: Class/ASD, Length: 704, dtype: object

In [12]:
y.value_counts()

NO     515
YES    189
Name: Class/ASD, dtype: int64

This shows that the dataset contains more NO to ASD than YES to ASD. From the analysis perspetive, this means we have an imbalanced dataset. 
So we should focus on the F1 score figure in our evaluation of a Machine Learning model.

In [13]:
X.describe()

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age
count,704.000000,704.000000,704.000000,704.000000,704.000000,704.000000,704.000000,704.000000,704.000000,704.000000,702.000000
mean,0.721591,0.453125,0.457386,0.495739,0.498580,0.284091,0.417614,0.649148,0.323864,0.573864,29.698006
std,0.448535,0.498152,0.498535,0.500337,0.500353,0.451301,0.493516,0.477576,0.468281,0.494866,16.507465
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,27.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,35.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,383.000000


In [14]:
# to find the max of Age, 383 years old, which is odd

# X.loc[ ( X.age == 383 ) ] or the below code
X.loc[ X['age'] == 383]

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,age_desc,relation
52,1,0,0,0,0,0,0,0,0,0,383.0,f,Pasifika,no,no,New Zealand,no,18 and more,Self


In [15]:
print(f"median of the Age column is {X.age.median()} \n" )
print( f"the outlier value of the Age is { X.at[52,'age'] } \n " )

X.at[52,'age'] = X.age.median()

print( f"the NEW outlier value of the Age is now the Median of the Age column, which is { X.at[52,'age'] } " )

median of the Age column is 27.0 

the outlier value of the Age is 383.0 
 
the NEW outlier value of the Age is now the Median of the Age column, which is 27.0 


## Converting text values in y into numeric values

In [16]:
from sklearn.preprocessing import LabelEncoder

y_label_encoder = LabelEncoder()

y_label_encoder.fit(y)
print( list(y_label_encoder.classes_) ) 

y_le_fit_transform = y_label_encoder.transform(y)
print("shape of the y modified = ", y_le_fit_transform.shape )

['NO', 'YES']
shape of the y modified =  (704,)


## Preprocessing the Training Set and Test Set

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

In [18]:
num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

In [19]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

cat_pipeline = Pipeline([
        ("ordinal_encoder", OrdinalEncoder()),    
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("cat_encoder", OneHotEncoder(sparse=False)),
    ])

In [20]:
num_attribs = X.select_dtypes(include=[np.number]).columns
num_attribs

Index(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score', 'A6_Score',
       'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score', 'age'],
      dtype='object')

In [21]:
cat_attribs = X.select_dtypes(include=[object]).columns
cat_attribs

Index(['gender', 'ethnicity', 'jundice', 'austim', 'contry_of_res',
       'used_app_before', 'age_desc', 'relation'],
      dtype='object')

In [22]:
from sklearn.compose import ColumnTransformer

preprocess = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

In [23]:
X_preprocess_fit_transform = preprocess.fit_transform(X)

X_preprocess_fit_transform.shape

(704, 103)

## Splitting dataset X,y into Training Set and Test Set

In [24]:
from sklearn.model_selection import train_test_split

strat_X_train_set, strat_X_test_set, strat_y_train_set, strat_y_test_set = train_test_split(
    X_preprocess_fit_transform, 
    y_le_fit_transform, 
    test_size = 0.2, 
    stratify = y, 
    random_state=42
)

In [25]:
print(f" strat_X_train_set shape is {strat_X_train_set.shape} ")
print(f" strat_y_train_set shape is {strat_y_train_set.shape} ")

 strat_X_train_set shape is (563, 103) 
 strat_y_train_set shape is (563,) 


In [26]:
print(f" strat_X_test_set shape is {strat_X_test_set.shape} ")
print(f" strat_y_test_set shape is {strat_y_test_set.shape} ")

 strat_X_test_set shape is (141, 103) 
 strat_y_test_set shape is (141,) 


## Fitting a model and Predicting the y values

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

In [28]:
clf_LogReg = LogisticRegression(random_state = 42, solver = 'lbfgs')
clf_DecTreeClf = DecisionTreeClassifier(random_state = 42)
clf_KNNClf = KNeighborsClassifier()
clf_AdaBoostClf = AdaBoostClassifier(random_state = 42)

classifier = [ clf_LogReg , clf_KNNClf, clf_DecTreeClf, clf_AdaBoostClf ]

label = ['Logistic Regression', 'K Neighbors Classifier', 'Decision Tree Classifier', 'AdaBoost Classifier']

fold = 10

In [29]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

## Performance of a model

In [30]:
for name, clf in zip(label, classifier):
    clf.fit( strat_X_train_set, strat_y_train_set )
    
    print( "Training Accuracy: {0: .6f} [ {1} is used. ]".format( 
        clf.score( strat_X_train_set, strat_y_train_set ), 
        name ) )

    print( "Test Accuracy: {0: .6f} [ {1} is used. ] \n ".format( 
        clf.score( strat_X_test_set, strat_y_test_set ), 
        name ) )

Training Accuracy:  1.000000 [ Logistic Regression is used. ]
Test Accuracy:  0.992908 [ Logistic Regression is used. ] 
 
Training Accuracy:  0.976909 [ K Neighbors Classifier is used. ]
Test Accuracy:  0.964539 [ K Neighbors Classifier is used. ] 
 
Training Accuracy:  1.000000 [ Decision Tree Classifier is used. ]
Test Accuracy:  0.851064 [ Decision Tree Classifier is used. ] 
 
Training Accuracy:  1.000000 [ AdaBoost Classifier is used. ]
Test Accuracy:  1.000000 [ AdaBoost Classifier is used. ] 
 


In [31]:
for name, clf in zip(label, classifier):
    
    scoresTrainSet = cross_val_score(
        estimator = clf, 
        X = strat_X_train_set,  
        y = strat_y_train_set, 
        scoring = 'accuracy', 
        cv = fold)
    
    print( "For {0}-fold cross-validation, Accuracy of Training Set: Mean = {1:.6f}, std = {2:.5f} [ {3} is used. ]".format( 
        fold,
        scoresTrainSet.mean(), 
        scoresTrainSet.std(), 
        name ) )
    
    scoresTestSet = cross_val_score(
        estimator = clf, 
        X = strat_X_test_set, 
        y = strat_y_test_set, 
        scoring = 'accuracy', 
        cv = fold)
    
    print( "For {0}-fold cross-validation, Accuracy of Test Set: Mean = {1:.6f}, std = {2:.5f} [ {3} is used. ] \n ".format( 
        fold,
        scoresTestSet.mean(), 
        scoresTestSet.std(), 
        name ) )
        
    print(37 * "<> ")
    print()

For 10-fold cross-validation, Accuracy of Training Set: Mean = 0.996429, std = 0.00714 [ Logistic Regression is used. ]
For 10-fold cross-validation, Accuracy of Test Set: Mean = 0.971429, std = 0.03499 [ Logistic Regression is used. ] 
 
<> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> 

For 10-fold cross-validation, Accuracy of Training Set: Mean = 0.960996, std = 0.02927 [ K Neighbors Classifier is used. ]
For 10-fold cross-validation, Accuracy of Test Set: Mean = 0.957619, std = 0.04711 [ K Neighbors Classifier is used. ] 
 
<> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> <> 

For 10-fold cross-validation, Accuracy of Training Set: Mean = 0.907613, std = 0.05126 [ Decision Tree Classifier is used. ]
For 10-fold cross-validation, Accuracy of Test Set: Mean = 0.836190, std = 0.09113 [ Decision Tree Classifier is used. ] 
 
<> <> <> <> <> <> <> <> <> <> <> <> <> <>

In [32]:
for name, clf in zip(label, classifier):
    predict_strat_y_train_set = clf.predict( strat_X_train_set )
    predict_strat_y_test_set = clf.predict( strat_X_test_set )
    
    clsF1ScoreTrainSet = f1_score(strat_y_train_set, predict_strat_y_train_set)
    print( "F1 Score for Training Set: {0: 6f} [ {1} is used. ] ".format( 
        clsF1ScoreTrainSet, name ) 
         )
    
    clsF1ScoreTestSet = f1_score(strat_y_test_set, predict_strat_y_test_set)
    print( "F1 Score for Test Set: {0: 6f} [ {1} is used. ] \n".format( 
        clsF1ScoreTestSet, name ) 
         )

F1 Score for Training Set:  1.000000 [ Logistic Regression is used. ] 
F1 Score for Test Set:  0.986667 [ Logistic Regression is used. ] 

F1 Score for Training Set:  0.957655 [ K Neighbors Classifier is used. ] 
F1 Score for Test Set:  0.936709 [ K Neighbors Classifier is used. ] 

F1 Score for Training Set:  1.000000 [ Decision Tree Classifier is used. ] 
F1 Score for Test Set:  0.720000 [ Decision Tree Classifier is used. ] 

F1 Score for Training Set:  1.000000 [ AdaBoost Classifier is used. ] 
F1 Score for Test Set:  1.000000 [ AdaBoost Classifier is used. ] 



## I am responsible any error in this analysis. If you spot an error please let me know. I also welcome any feedback. Thank you for your time and attention.